In [1]:
#export
"""
This is for all short and random quality-of-life utilities."""
from k1lib.cli.init import patchDefaultDelim, BaseCli, yieldT
import k1lib.cli as cli, k1lib.cli.init as init, numbers, numpy as np, dis
from k1lib.cli.typehint import *
from typing import overload, Iterator, Any, List, Set, Union, Callable
import k1lib, time, math, os, json
from collections import defaultdict
try: import torch; hasTorch = True
except: torch = k1lib.Object().withAutoDeclare(lambda: type("RandomClass", (object, ), {})); hasTorch = False
try: import PIL; hasPIL = True
except: hasPIL = False
try: plt = k1lib.dep("matplotlib.pyplot")
except: pass
__all__ = ["size", "shape", "item", "rItem", "iden", "join", "wrapList",
           "equals", "reverse", "ignore", "rateLimit", "timeLimit", "tab", "indent",
           "clipboard", "deref", "bindec", "smooth", "disassemble",
           "tree", "lookup", "dictFields", "backup", "sketch", "syncStepper", "zeroes", "normalize"]

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
cli.init.patchNumpy()

In [3]:
#export
settings = k1lib.settings.cli

In [4]:
#export
def exploreSize(it):
    """Returns first element and length of array. Returns [first item, length]"""
    if isinstance(it, str): return None, len(it)
    try: return it[0], len(it)
    except: pass
    sentinel = object(); it = iter(it)
    o = next(it, sentinel); count = 1
    if o is sentinel: return None, 0
    try:
        while True: next(it); count += 1
    except StopIteration: pass
    return o, count

In [5]:
#export
class size(BaseCli):
    def __init__(self, idx=None):
        """Returns number of rows and columns in the input.
Example::

    # returns (3, 2)
    [[2, 3], [4, 5, 6], [3]] | shape()
    # returns 3
    [[2, 3], [4, 5, 6], [3]] | shape(0)
    # returns 2
    [[2, 3], [4, 5, 6], [3]] | shape(1)
    # returns (2, 0)
    [[], [2, 3]] | shape()
    # returns (3,)
    [2, 3, 5] | shape()
    # returns 3
    [2, 3, 5] | shape(0)
    # returns (3, 2, 2)
    [[[2, 1], [0, 6, 7]], 3, 5] | shape()
    # returns (1, 3)
    ["abc"] | shape()
    # returns (1, 2, 3)
    [torch.randn(2, 3)] | shape()
    # returns (2, 3, 5)
    shape()(np.random.randn(2, 3, 5))

:class:`shape` is an alias of this cli. Use whichever is more intuitive for you.

There's also :class:`lengths`, which is sort of a simplified/faster version of
this, but only use it if you are sure that ``len(it)`` can be called.

:param idx: if not specified, returns a tuple of ints. If specified,
    then returns the specific index of the tuple"""
        super().__init__(); self.idx = idx;
        if idx is not None: self._f = cli.item(idx)
    def _all_array_opt(self, it, level): return np.array(it.shape[level:])[tuple([None]*level)] + np.zeros(it.shape[:level], dtype=int)[(*[slice(None)]*level, None)]
    def _typehint(self, inp):
        if self.idx is not None: return int
        return tList(int)
    def __ror__(self, it:Iterator[str]):
        idx = self.idx
        if idx == 0:
            try: return len(it)
            except:
                try: return exploreSize(it)[1]
                except: pass
        if hasPIL and isinstance(it, PIL.Image.Image):
            return it.size if idx is None else it.size[idx]
        if idx is None:
            answer = []
            try:
                while True:
                    if isinstance(it, settings.arrayTypes):
                        return tuple(answer + list(it.shape))
                    it, s = exploreSize(it); answer.append(s)
            except TypeError: pass
            return tuple(answer)
        return exploreSize(it | self._f)[1]
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        post = "" if self.idx is None else f"[{cli.kjs.v(self.idx)}]"
        return f"const {fIdx} = ({dataIdx}) => {dataIdx}.shape(){post}", fIdx
shape = size

In [6]:
assert [[2, 3], [4, 5, 6], [3]] | tCheck() | size() == (3, 2)
assert [[2, 3], [4, 5, 6], [3]] | tCheck() | size(0) == 3
assert [[2, 3], [4, 5, 6], [3]] | tCheck() | size(1) == 2
assert [[], [2, 3]] | tCheck() | size() == (2, 0)
assert [2, 3, 5] | tCheck() | size() == (3,)
assert [2, 3, 5] | tCheck() | size(0) == 3
assert torch.randn(3, 4) | tCheck() | size() == (3, 4)
assert [[[2, 1], [0, 6, 7]], 3, 5] | tCheck() | size() == (3, 2, 2)
assert ["abc"] | tCheck() | size() == (1, 3)
assert [torch.randn(2, 3)] | tCheck() | size() == (1, 2, 3)
assert size()(np.random.randn(2, 3, 5)) | tCheck() == (2, 3, 5)
assert np.random.randn(3, 4, 5, 6) | shape().all(2) | cli.op().shape == (3, 4, 2)

In [7]:
#export
noFill = object()
class item(BaseCli):
    def __init__(self, amt:int=1, fill=noFill):
        """Returns the first element of the input iterator.
Example::

    # returns 0
    range(5) | item()
    # returns torch.Size([5])
    torch.randn(3,4,5) | item(2) | shape()
    # returns 3
    [] | item(fill=3)

:param amt: how many times do you want to call item() back to back?
:param fill: if iterator length is 0, return this"""
        self.amt = amt; self.fill = fill
        self.fillP = [fill] if fill != noFill else [] # preprocessed, to be faster
        if self.amt != 1: self._f = cli.serial(*(item(fill=self.fill) for _ in range(self.amt)))
    def _all_array_opt(self, it, level): return it[(*[slice(None, None, None) for i in range(level)], 0)]
    def _typehint(self, inp):
        if isinstance(inp, tListIterSet): return inp.child
        if isinstance(inp, tCollection): return inp.children[0]
        if isinstance(inp, tArrayTypes):
            if inp.rank is None: return inp.__class__(inp.child, None)
            if inp.rank - self.amt >= 1: return inp.__class__(inp.child, inp.rank-self.amt)
            return inp.child
        return tAny()
    def __ror__(self, it:Iterator[str]):
        if self.amt != 1: return it | self._f
        return next(iter(it), *self.fillP)
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"const {fIdx} = ({dataIdx}) => {dataIdx}[0]", fIdx

In [8]:
assert iter(range(5)) | tCheck() | item() == 0
assert torch.randn(3,4,5) | tCheck() | item(2) | shape() == torch.Size([5])
assert [] | item(fill=3) == 3
assert "test/crew dragon.jpg" | cli.toImg() | shape() == (1280, 853)

In [9]:
#export
class rItem(BaseCli):
    def __init__(self, idx:int):
        """Combines ``rows(idx) | item()``, as this is a pretty common pattern.
Example::

    iter(range(10)) | rItem(4) # returns 4
"""
        self.idx = idx; self.arrayTypes = (*settings.arrayTypes, list, tuple)
    def _all_array_opt(self, it, level:int): return it[(*[slice(None, None, None) for i in range(level)], self.idx)]
    def __ror__(self, it):
        if isinstance(it, self.arrayTypes): return it[self.idx]
        for i, e in zip(range(self.idx+1), it): pass
        return e
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"const {fIdx} = ({dataIdx}) => {dataIdx}[{cli.kjs.v(self.idx)}]", fIdx

In [10]:
assert iter(range(10)) | rItem(0) == 0
assert iter(range(10)) | rItem(4) == 4
assert iter(range(10)) | rItem(9) == 9
assert np.random.randn(3, 4, 5, 6) | rItem(1) | cli.op().shape == (4, 5, 6)
assert np.random.randn(3, 4, 5, 6) | rItem(1).all() | cli.op().shape == (3, 5, 6)

In [11]:
#export
class iden(BaseCli):
    def __init__(self):
        """Yields whatever the input is. Useful for multiple streams.
Example::

    # returns range(5)
    range(5) | iden()"""
        super().__init__()
    def _all_array_opt(self, it, level): return it
    def _typehint(self, inp): return inp
    def __ror__(self, it:Iterator[Any]): return it
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"const {fIdx} = ({dataIdx}) => {dataIdx}", fIdx

In [12]:
assert range(5) | tCheck() | iden() == range(5)

In [13]:
#export
class join(BaseCli):
    def __init__(self, delim:str=None):
        r"""Merges all strings into 1, with `delim` in the middle. Basically
:meth:`str.join`. Example::

    # returns '2\na'
    [2, "a"] | join("\n")"""
        super().__init__(); self.delim = patchDefaultDelim(delim)
    def _typehint(self, inp): return str
    def __ror__(self, it:Iterator[str]):
        return self.delim.join(it | cli.apply(str))
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"const {fIdx} = ({dataIdx}) => {dataIdx}.join({json.dumps(self.delim)})", fIdx

In [14]:
assert [2, "a"] | tCheck() | join("\n") == '2\na'

In [15]:
#export
class wrapList(BaseCli):
    def __init__(self):
        """Wraps inputs inside a list. There's a more advanced cli tool
built from this, which is :meth:`~k1lib.cli.structural.unsqueeze`. Example::

    # returns [5]
    5 | wrapList()"""
        super().__init__()
    def _all_array_opt(self, it, level): return it[(*[slice(None)]*level, None)]
    def _typehint(self, inp): return tList(inp)
    def __ror__(self, it) -> List[Any]:
        if isinstance(it, settings.arrayTypes): return it[None]
        return [it]
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"const {fIdx} = ({dataIdx}) => [{dataIdx}]", fIdx

In [16]:
assert np.random.randn(3, 4, 5) | wrapList().all() | cli.op().shape == (3, 1, 4, 5)
assert np.random.randn(3, 4, 5) | wrapList() | cli.op().shape == (1, 3, 4, 5)

In [17]:
#export
class _EarlyExp(Exception): pass
class equals:
    def __init__(self):
        """Checks if all incoming columns/streams are identical"""
        super().__init__()
    def __ror__(self, streams:Iterator[Iterator[str]]):
        streams = list(streams)
        for row in zip(*streams):
            sampleElem = row[0]
            try:
                for elem in row:
                    if sampleElem != elem: yield False; raise _EarlyExp()
                yield True
            except _EarlyExp: pass

In [18]:
range(10)[slice(None, None, -1)] | cli.deref()

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [19]:
#export
class reverse(BaseCli):
    def __init__(self):
        """Reverses incoming list.
Example::

    # returns [3, 5, 2]
    [2, 5, 3] | reverse() | deref()"""
        super().__init__()
    def _all_array_opt(self, it, level): return it[(*[slice(None)]*level, slice(None, None, -1))]
    def _typehint(self, inp):
        if isinstance(inp, tListIterSet): return tIter(inp.child)
        return tAny()
    def __ror__(self, it:Iterator[str]) -> List[str]:
        if isinstance(it, settings.arrayTypes): return it[::-1]
        return reversed(list(it))
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"const {fIdx} = ({dataIdx}) => [...{dataIdx}].reverse()", fIdx

In [20]:
assert [2, 5, 3] | tCheck() | reverse() | cli.deref() == [3, 5, 2]
assert np.random.randn(3, 4, 5) | reverse().all() | cli.op().shape == (3, 4, 5)

In [21]:
#export
class ignore(BaseCli):
    def __init__(self):
        r"""Just loops through everything, ignoring the output.
Example::

    # will just return an iterator, and not print anything
    [2, 3] | apply(lambda x: print(x))
    # will prints "2\n3"
    [2, 3] | apply(lambda x: print(x)) | ignore()"""
        super().__init__()
    def _all_array_opt(self, it, level): return it
    def _typehint(self, inp): return type(None)
    def __ror__(self, it:Iterator[Any]):
        if isinstance(it, settings.arrayTypes): return
        for _ in it: pass
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"const {fIdx} = ({dataIdx}) => {dataIdx}", fIdx

In [22]:
with k1lib.captureStdout() as out:
    [2, 3] | cli.apply(lambda x: print(x))
assert len(out()) == 1
with k1lib.captureStdout() as out:
    [2, 3] | tCheck() | cli.apply(lambda x: print(x)) | ignore()
assert out() == ["2", "3", '']

In [23]:
#export
class rateLimit(BaseCli):
    def __init__(self, f, delay=0.1):
        """Limits the execution flow rate upon a condition.
Example::

    s = 0; semaphore = 0
    def heavyAsyncOperation(i):
        global semaphore, s
        semaphore += 1
        s += i; time.sleep(1)
        semaphore -= 1; return i**2

    # returns (20,), takes 1s to run
    range(20) | applyTh(heavyAsyncOperation, 100) | shape()
    # returns (20,), takes 4s to run (20/5 = 4)
    range(20) | rateLimit(lambda: semaphore < 5) | applyTh(heavyAsyncOperation, 100) | shape()

The first test case is not rate-limited, so it will run all 20 threads at the
same time, and all of them will finish after 1 second.

The second test case is rate-limited, so that there can only be 5 concurrently
executing threads because of the semaphore count check. Therefore this takes
around 4 seconds to run.

:param f: checking function. Should return true if execution is allowed
:param delay: delay in seconds between calling ``f()``"""
        self.f = f; self.delay = delay
    def _typehint(self, inp):
        if isinstance(inp, tListIterSet): return tIter(inp.child)
        if isinstance(inp, tArrayTypes):
            if inp.rank is None: return tIter(inp)
            if inp.rank >= 2: return tIter(inp.__class__(inp.child, inp.rank - 1))
            return tIter(inp.child)
        if isinstance(inp, tCollection): return inp
        return tAny()
    def __ror__(self, it):
        f = self.f; delay = self.delay
        for e in it:
            while not f(): time.sleep(delay)
            yield e
    @staticmethod
    def cpu(maxUtilization=90):
        """Limits flow rate when cpu utilization is more than a specified
percentage amount. Needs to install the package ``psutil`` to actually work.
Example::

    # returns [0, 1, 4, 9, 16]
    range(5) | rateLimit.cpu() | apply(op()**2) | deref()"""
        import psutil
        return rateLimit(lambda: psutil.cpu_percent() < maxUtilization)

In [24]:
s = 0; semaphore = 0
def heavyAsyncOperation(i):
    global semaphore, s
    semaphore += 1
    s += i; time.sleep(1)
    semaphore -= 1; return i**2
with k1lib.timer() as t:
    assert range(20) | rateLimit(lambda: semaphore < 5) | cli.applyTh(heavyAsyncOperation, 100) | cli.shape() == (20,)
assert 3.8 < t() < 4.2
with k1lib.timer() as t:
    assert range(20) | cli.applyTh(heavyAsyncOperation, 100) | cli.shape() == (20,)
assert 0.8 < t() < 1.2
assert range(5) | rateLimit.cpu() | cli.apply(cli.op()**2) | cli.deref() == [0, 1, 4, 9, 16]

In [25]:
#export
class timeLimit(BaseCli):
    def __init__(self, t):
        """Caps the flow after a specified amount of time has
passed. Example::

    # returns 20, or roughly close to that
    repeatF(lambda: time.sleep(0.1)) | timeLimit(2) | shape(0)"""
        self.t = t
    def _typehint(self, inp):
        if isinstance(inp, tListIterSet): return tIter(inp.child)
        if isinstance(inp, tArrayTypes):
            if inp.rank is None: return tIter(inp)
            if inp.rank >= 2: return tIter(inp.__class__(inp.child, inp.rank - 1))
            return tIter(inp.child)
        if isinstance(inp, tCollection): return inp
        return tAny()
    def __ror__(self, it):
        _time = time.time; endTime = _time() + self.t
        for e in it:
            yield e
            if _time() > endTime: break

In [26]:
assert cli.repeatF(lambda: time.sleep(0.1)) | timeLimit(2) | shape(0) | cli.op() - 20 | cli.aS(abs) | cli.op() < 2

In [27]:
#export
def tab(pad:str=" "*4):
    """Indents incoming string iterator.
Example::

    # prints out indented 0 to 9
    range(10) | tab() | headOut()"""
    return cli.apply(lambda x: f"{pad}{x}")
indent = tab

In [28]:
assert range(2) | tab() | cli.join("|") == '    0|    1'

In [29]:
#export
class clipboard(BaseCli):
    def __init__(self):
        """Saves the input to clipboard.
Example::

    # copies "abc" into the clipboard. Just use Ctrl+V to paste as usual
    "abc" | clipboard()"""
        import pyperclip; self.pyperclip = pyperclip
    def _typehint(self, inp): return type(None)
    def __ror__(self, s): self.pyperclip.copy(s)

In [30]:
#assert "def2r" | tCheck() | clipboard() == None

In [31]:
#export
a = [numbers.Number, np.number, str, bool, bytes, k1lib.UValue, cli.conv.Audio]
if hasTorch: a.append(torch.nn.Module)
settings.atomic.add("deref", tuple(a), "used by deref")
Tensor = torch.Tensor; atomic = settings.atomic
class inv_dereference(BaseCli):
    def __init__(self, igT=False):
        """Kinda the inverse to :class:`dereference`"""
        super().__init__(); self.igT = igT
    def __ror__(self, it:Iterator[Any]) -> List[Any]:
        for e in it:
            if e is None or isinstance(e, atomic.deref): yield e
            elif isinstance(e, settings.arrayTypes):
                if not self.igT and len(e.shape) == 0: yield e.item()
                else: yield e
            else:
                try: yield e | self
                except: yield e

In [32]:
#export
class deref(BaseCli):
    def __init__(self, maxDepth=float("inf"), igT=True):
        """Recursively converts any iterator into a list.
Example::

    # returns something like "<range_iterator at 0x7fa8c52ca870>"
    iter(range(5))
    # returns [0, 1, 2, 3, 4]
    iter(range(5)) | deref()
    # returns [2, 3], yieldT stops things early
    [2, 3, yieldT, 6] | deref()

You can also specify a ``maxDepth``::

    # returns something like "<list_iterator at 0x7f810cf0fdc0>"
    iter([range(3)]) | deref(0)
    # returns [range(3)]
    iter([range(3)]) | deref(1)
    # returns [[0, 1, 2]]
    iter([range(3)]) | deref(2)

There are a few classes/types that are considered atomic, and :class:`deref`
will never try to iterate over it. If you wish to change it, do something like::

    settings.cli.atomic.deref = (int, float, ...)

:param maxDepth: maximum depth to dereference. Starts at 0 for not doing anything
    at all
:param igT: short for "ignore tensor". If True, then don't loop over :class:`torch.Tensor`
    and :class:`numpy.ndarray` internals"""
        super().__init__(); self.igT = igT
        self.maxDepth = maxDepth; self.depth = 0
        if hasTorch:
            self.arrayType = (torch.Tensor, np.ndarray) if k1lib.settings.startup.or_patch.numpy else torch.Tensor
        else: self.arrayType = (np.ndarray,) if k1lib.settings.startup.or_patch.numpy else ()
    def _typehint(self, inp, depth=float("inf")):
        if depth == 0: return inp
        if depth == float("inf"): depth = self.maxDepth
        if isinstance(inp, type) and issubclass(inp, atomic.deref): return inp
        if isinstance(inp, tArrayTypes):
            if self.igT: return inp
            if inp.rank is None: return tList(tAny())
            if inp.rank == 1:
                if isinstance(inp, tTensor):
                    return tList(type(torch.tensor(3, dtype=inp.child).item()))
                if isinstance(inp, tNpArray):
                    return tList(type(np.array(3, dtype=inp.child).item()))
            return tList(self._typehint(inp.item(), depth-1))
        if isinstance(inp, tListIterSet):
            return tList(self._typehint(inp.child, depth-1))
        if isinstance(inp, tCollection):
            return tCollection(*(self._typehint(e, depth-1) for e in inp.children))
        return tAny()
    def __ror__(self, it:Iterator[Any]) -> List[Any]:
        if self.depth >= self.maxDepth: return it
        elif isinstance(it, np.number): return it.item()
        elif isinstance(it, atomic.deref): return it
        elif isinstance(it, self.arrayType):
            if self.igT: return it
            if len(it.shape) == 0: return it.item()
        elif isinstance(it, dict):  self.depth += 1; _d = {k:   self.__ror__(v) for k, v in it.items()}; self.depth -= 1; return _d
        elif isinstance(it, tuple): self.depth += 1; _t = tuple(self.__ror__(k) for k    in it);         self.depth -= 1; return _t
        elif isinstance(it, set):   self.depth += 1; _s = set  (self.__ror__(k) for k    in it);         self.depth -= 1; return _s
        try: iter(it)
        except: return it
        self.depth += 1; answer = []
        for e in it:
            if e is cli.yieldT: return answer
            answer.append(self.__ror__(e))
        self.depth -= 1; return answer
    def __invert__(self) -> BaseCli:
        """Returns a :class:`~k1lib.cli.init.BaseCli` that makes
everything an iterator. Not entirely sure when this comes in handy, but it's
there."""
        return inv_dereference(self.igT)
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"const {fIdx} = ({dataIdx}) => {dataIdx}", fIdx
    @staticmethod
    def js():
        """Deref incoming object and turn them into a js object (NOT json string!).
Example::

    # returns "[...Array(10).keys()]"
    range(10) | deref.json()

How does it know to transpile it? Based on the dictionary at `settings.cli.kjs.jsonF`
and the object's "._jsonF" function. Say you have a custom list object, you can do
something like this::

    class CustomList:
        def __init__(self): ...
        def _jsonF(self): return "your js string here"

Or, you can do something like this::

    class CustomList: ...
    settings.cli.kjs.jsonF[CustomList] = lambda obj: "your js string here"

A variety of data types are included out of the box already for common types,
view the source code of this method to check them out."""
        jsonF = settings.kjs.jsonF
        jsonF[list] = lambda x: "[" + ", ".join([deref_js(e) for e in x]) + "]"
        jsonF[str] = lambda x: json.dumps(x)
        jsonF[tuple] = jsonF[list]; jsonF[set] = lambda x: "new Set(" + jsonF[list](x) + ")"
        jsonF[type(None)] = lambda x: "null"
        jsonF[np.ndarray] = lambda x: json.dumps(x | deref(igT=False))
        if hasTorch: jsonF[torch.Tensor] = lambda x: json.dumps(x | deref(igT=False))
        jsonF[type(iter(range(10)))] = lambda x: "[" + ", ".join([str(e) for e in x]) + "]"
        jsonF[type((x for x in range(0)))] = jsonF[list]
        jsonF[type({}.keys())] = jsonF[list]; jsonF[type({}.values())] = jsonF[list]
        jsonF[dict] = lambda x: "{" + ", ".join([f"{json.dumps(k)}: {deref_js(v)}" for k,v in x.items()]) + "}"
        jsonF[defaultdict] = jsonF[dict]
        deref.js = lambda: cli.aS(deref_js); return deref.js() # initializes at runtime, then patches deref.json() to get a faster path!

In [33]:
import numpy as np, numbers, torch
a = torch.linspace(0, 10, 50) | deref(igT=False)
b = torch.from_numpy(np.linspace(0, 10, 50)) | deref(igT=False)
assert torch.allclose(torch.tensor(b), torch.tensor(a))
assert iter([range(3)]) | tCheck() | deref(1) == [range(3)]
assert iter([range(3)]) | tCheck() | deref(2) == [[0, 1, 2]]
assert torch.randn(2, 3) | tCheck() | deref(igT=False) | ~deref() | deref() | shape() == (2, 3)
assert [2, 3, cli.yieldT, 6] | deref() == [2, 3]
with k1lib.settings.cli.atomic.context(deref=(int, float, ...)): assert k1lib.settings.cli.atomic.deref == (int, float, ...)
assert np.array([[1, 2, 3], [4, 5, 6]]) | tCheck() | deref(igT=False) == [[1, 2, 3], [4, 5, 6]]
assert {"a": range(3)} | tCheck() | deref() == {'a': [0, 1, 2]}
assert deref(1, igT=False)._typehint(tTensor(float, 2)) == tList(tTensor(float, 1))
assert deref(1, igT=False)._typehint(tTensor(float, 2)) != tList(tTensor(float, 2))
assert deref(igT=False)._typehint(tTensor(float, 2)) == tList(tList(float))
assert deref()._typehint(tList(tList(int))) == tList(tList(int))

In [34]:
#export
def deref_js(obj):
    # only 2 special cases, perf considerations, everything else is pluggable
    if isinstance(obj, bool): return "true" if obj else "false"
    if isinstance(obj, (numbers.Number, np.number)): return str(obj)
    fn = settings.kjs.jsonF.get(type(obj), None)
    if fn: return fn(obj)
    if hasattr(obj, "_jsonF"): return obj._jsonF()
    raise Exception(f"Don't know how to transcribe object with class {type(obj)}. Either add the serialization function to `settings.cli.kjs.jsonF`, or implement the function `._jsonF()` to your custom class")

In [35]:
def g(): yield 4; yield 2
assert [range(10), 4, set(range(10)), None, np.linspace(2, 10, 5), iter(range(5)), g(), "abc", '"abc"', True, {"a": 3, "b": [3, 4]}] | deref.js() == '[[...Array(10).keys()], 4, new Set([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), null, [2.0, 4.0, 6.0, 8.0, 10.0], [0, 1, 2, 3, 4], [4, 2], "abc", "\\"abc\\"", true, {"a": 3, "b": [3, 4]}]'

In [34]:
#export
class bindec(BaseCli):
    def __init__(self, cats:List[Any], f=None):
        """Binary decodes the input.
Example::

    # returns ['a', 'c']
    5 | bindec("abcdef")
    # returns 'a,c'
    5 | bindec("abcdef", join(","))

:param cats: categories
:param f: transformation function of the selected elements. Defaulted to :class:`toList`, but others like :class:`join` is useful too"""
        self.cats = cats; self.f = f or cli.toList()
    def __ror__(self, it):
        it = bin(int(it))[2:][::-1]
        return (e for i, e in zip(it, self.cats) if i == '1') | self.f

In [35]:
assert 5 | bindec("abcdef") == ['a', 'c']
assert 5 | bindec("abcdef", join(",")) == "a,c"

In [36]:
#export
settings.add("smooth", 10, "default smooth amount, used in utils.smooth")
class smooth(BaseCli):
    def __init__(self, consecutives=None):
        """Smoothes out the input stream.
Literally just a shortcut for::

    batched(consecutives) | toMean().all()

Example::

    # returns [4.5, 14.5, 24.5]
    range(30) | smooth(10) | deref()

Smoothing over :class:`torch.Tensor` or :class:`numpy.ndarray` will
be much faster, and produce high dimensional results::

    # returns torch.Tensor with shape (2, 3, 4)
    torch.randn(10, 3, 4) | smooth(4)

The default consecutive value is in ``settings.cli.smooth``. This
is useful if you are smoothing over multiple lists at the same
time, like this::

    # can change a single smooth value temporarily here, and all sequences will be smoothed in the same way
    with settings.cli.context(smooth=5):
        x = list(np.linspace(-2, 2, 50))
        y = x | apply(op()**2) | deref()
        plt.plot(x | smooth() | deref(), y | smooth() | deref())

:param consecutives: if not defined, then used the value inside ``settings.cli.smooth``"""
        self.b = cli.batched(consecutives or settings.smooth); self.consecutives = consecutives
    def __ror__(self, it):
        it = it | self.b
        if isinstance(it, settings.arrayTypes): return it.mean(1)
        return it | cli.toMean().all()
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto()
        return f"const {fIdx} = ({dataIdx}) => {dataIdx}.smooth({cli.kjs.v(self.consecutives)})", fIdx

In [37]:
assert range(30) | smooth(10) | deref() == [4.5, 14.5, 24.5]
assert range(30) | smooth() | deref() == [4.5, 14.5, 24.5]
with settings.context(smooth=5):
    x = list(np.linspace(-2, 2, 50))
    y = x | cli.apply(cli.op()**2) | deref()
    assert x | smooth() | shape(0) == 10
    assert y | smooth() | shape(0) == 10
assert torch.randn(10, 3, 4) | smooth(4) | shape() | cli.op() == (2, 3, 4)

In [38]:
#export
def _f(): pass
_code = type(_f.__code__)
def disassemble(f=None):
    """Disassembles anything piped into it.
Normal usage::

    def f(a, b):
        return a**2 + b
    # both of these print out disassembled info
    f | disassemble()
    disassemble(f)
    
    # you can pass in lambdas
    disassemble(lambda x: x + 3)
    
    # or even raw code
    "lambda x: x + 3" | disassemble()"""
    c = f
    if c is None: return cli.aS(disassemble)
    if isinstance(c, str): c = compile(c, "", "exec")
    try: c = c.__code__
    except: pass
    if not isinstance(c, _code): raise RuntimeError(f"`{c}` is not a code object/function/class method/string code")
    print(f"co_argcount: {c.co_argcount}")
    print(f"co_cellvars: {c.co_cellvars}")
    print(f"co_consts: {c.co_consts}")
    print(f"co_filename: {c.co_filename}")
    print(f"co_firstlineno: {c.co_firstlineno}")
    print(f"co_flags: {c.co_flags}")
    print(f"co_freevars: {c.co_freevars}")
    print(f"co_kwonlyargcount: {c.co_kwonlyargcount}")
    print(f"co_lnotab: {c.co_lnotab | cli.apply(str) | join(' ')}")
    print(f"co_name: {c.co_name}")
    print(f"co_names: {c.co_names}")
    print(f"co_nlocals: {c.co_nlocals}")
    print(f"co_posonlyargcount: {c.co_posonlyargcount}")
    print(f"co_stacksize: {c.co_stacksize}")
    print(f"co_varnames: {c.co_varnames}")
    print(f"Disassembly:"); dis.disassemble(c)
    with k1lib.captureStdout() as out:
        c.co_consts | cli.filt(lambda x: "code" in str(type(x))) | cli.tee(lambda _: "----------------------- inner code object -----------------------\n") | cli.apply(disassemble) | cli.ignore()
    out() | cli.filt(cli.op().strip() != "") | cli.apply("|" + cli.op()) | cli.indent() | cli.stdout()

In [39]:
def f(a, b):
    return a**2 + b
with k1lib.captureStdout() as out:
    f | disassemble(); disassemble(f);
    "lambda x: x + 3" | disassemble()
    disassemble(lambda x: x + 3)
assert out() | cli.shape(0) > 0

In [40]:
#export
shortName = lambda s: s.split(os.sep)[-1]
def tree(fL=10, dL=10, depth=float("inf"), ff:Callable[[str], bool]=(lambda s: True), df:Callable[[str], bool]=(lambda s: True)):
    """Recursively gets all files and folders. Output format might be a bit
strange, so this is mainly for visualization. Example::

    "." | tree() | deref()

:param fL: max number of file per directory included in output
:param dL: max number of child directories per directory included in output
:param depth: explore depth
:param ff: optional file filter function
:param df: optional directory filter function"""
    processFolders = cli.apply(lambda x: [shortName(x), x]) | cli.apply(lambda x: x | tree(fL, dL, depth-1, ff, df) if depth > 0 else [], 1) | cli.toDict()
    a = cli.filt(os.path.isfile) | cli.filt(ff) | cli.head(fL) | cli.apply(shortName) | cli.aS(set)
    b = ~cli.filt(os.path.isfile) | cli.filt(df) | cli.head(dL) | processFolders
    return cli.ls() | ~cli.sortF(os.path.isfile) | (a & b)

In [41]:
assert "." | tree() | shape() | shape(0) > 0

In [164]:
#export
class lookup(BaseCli):
    def __init__(self, d:dict, col:int=None, fill=None):
        """Looks up items from a dictionary/object. Example::

    d = {"a": 3, "b": 5, "c": 52}
    # returns [3, 5, 52, 52, 3]
    "abcca" | lookup(d) | deref()

    # raises Exception, as "d" does not exist
    "abccad" | lookup(d) | deref()
    # returns [3, 5, 52, 52, 3, '(not found)']
    "abccad" | lookup(d, fill="(not found)") | deref()
    # returns [3, 5, 52, 52, 3, 'd']
    "abccad" | lookup(d, fill=input) | deref()

    # returns [[0, 3], [1, 5], [2, 52], [3, 52], [4, 3]]
    [range(5), "abcca"] | transpose() | lookup(d, 1) | deref()

:param d: any object that can be sliced with the inputs
:param col: if None, lookup on each row, else lookup a specific
    column only
:param fill: if None, throws error if looked up element is not
    available, else returns the fill value (or the input value if ``fill == input``)"""
        self.d = d; self.col = col; self.fill = fill
        if fill is None: self.f = lambda e: d[e]
        elif fill == input: self.f = lambda e: d.get(e, e)
        else: self.f = lambda e: d.get(e, fill)
    def _typehint(self, inp):
        t = inferType(list(self.d.values()))
        if isinstance(t, tListIterSet): return tIter(t.child)
        if isinstance(t, tCollection): return tIter(tLowest(*t.children))
        return tIter(tAny())
    def __ror__(self, it): return it | cli.apply(self.f, self.col)
    def _jsF(self, meta):
        fIdx = init._jsFAuto(); dictIdx = init._jsDAuto(); dataIdx = init._jsDAuto()
        if not self.col is None: raise Exception("lookup._jsF() doesn't support custom .col yet")
        return f"const {dictIdx} = {json.dumps(self.d)}\nconst {fIdx} = ({dataIdx}) => {dataIdx}.lookup({dictIdx}, {cli.kjs.v(self.col)}, {cli.kjs.v(self.fill)})", fIdx

In [175]:
d = {"a": 3, "b": 5, "c": 52}
assert "abcca" | lookup(d) | deref() == [3, 5, 52, 52, 3]
try: "abccad" | lookup(d) | deref(); raise Exception("Failed")
except KeyError: pass
assert "abccad" | lookup(d, fill="(not found)") | deref() == [3, 5, 52, 52, 3, '(not found)']
assert "abccad" | lookup(d, fill=input) | deref() == [3, 5, 52, 52, 3, 'd']
assert [range(5), "abcca"] | cli.transpose() | lookup(d, 1) | deref() == [[0, 3], [1, 5], [2, 52], [3, 52], [4, 3]]
assert lookup(d)._typehint(3) == tIter(int)

In [44]:
#export
class dictFields(BaseCli):
    def __init__(self, *fields, default=""):
        """Grab a bunch of dictionary fields.
Example::

    # returns [3, 1, '']
    {"a": 1, "b": 2, "c": 3} | dictFields("c", "a", "d")
"""
        self.fields = fields; self.default = default
    def __ror__(self, d):
        return [d.get(f, self.default) for f in self.fields]

In [45]:
assert {"a": 1, "b": 2, "c": 3} | dictFields("c", "a", "d") == [3, 1, '']

In [46]:
#export
class backup(BaseCli):
    def __init__(self):
        """Backs up a file/folder.
Example::

    "some/folderOrFile" | backup()
    "some/folderOrFile" | backup.restore()

Really straightforward. Uses bash internally to copy files recursively, so
not available on Windows."""
        pass
    def __ror__(self, it):
        it = os.path.expanduser(it)
        None | cli.cmd(f"rm -rf '{it}.backup'") | cli.ignore()
        None | cli.cmd(f"cp -r '{it}' '{it}.backup'") | cli.ignore()
    @staticmethod
    def restore():
        def inner(it):
            it = os.path.expanduser(it)
            None | cli.cmd(f"rm -rf '{it}'") | cli.ignore()
            None | cli.cmd(f"cp -r '{it}.backup' '{it}'") | cli.ignore()
        return cli.aS(inner)

In [47]:
#export
sketch_interceptor = {}
class sketch(BaseCli):
    _jsF_ctxIdx = None
    def __init__(self, transforms:List[callable]=[], titles:List[str]=None, im:bool=False, ncols:int=None, n:int=None, axes:int=None):
        """Convenience tool to plot multiple matplotlib plots at the same
time, while still keeping everything short and in 1 line. For this example,
we're trying to plot x^1, x^2, ..., x^8 on 2 separate plots, one left one
right. The left will have x^1 till x^4, the right will have x^5 to x^8.

How you would do this normally::

    x = np.linspace(-2, 2); exps = range(1, 9)

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    # simplest solution
    plt.sca(axes[0]); plt.plot(x, x**1); plt.plot(x, x**2); plt.plot(x, x**3); plt.plot(x, x**4); plt.legend([1, 2, 3, 4]); plt.xlabel("x axis")
    # solution using a little bit of cli
    plt.sca(axes[1]); range(5, 9) | apply(lambda a: [x, x**a]) | ~apply(plt.plot) | ignore();     plt.legend([5, 6, 7, 8]); plt.xlabel("x axis")

But this is long, and I'm incredibly lazy to write it all out. So here's how
it's going to work using this cli::

    # plotting the first 4 lines only, in a single plot. Should be familiar and make sense to you before moving on
    exps | apply(lambda a: [x, x**a]) | batched(4) | item() | ~apply(plt.plot) | ignore()

    # plotting 8 lines across 2 plots. Simplest example using sketch(). It kinda captures clis after it and use it to plot each plot
    exps | apply(lambda a: [x, x**a]) | batched(4) | (sketch() | ~apply(plt.plot))

    # same as above, but adding a grid and x axis label to all plots. Transformation functions can be anything you would
    # put inside a normal cli (`plt` will be passed as argument): string code, op()-capture, lambda functions, other cli tools
    transforms = ["x.grid(True)", op().xlabel("x axis"), lambda x: x.ylabel("y axis")]
    exps | apply(lambda a: [x, x**a]) | batched(4) | (sketch(transforms) | ~apply(plt.plot))

    # same as above, but adding legends. [x, x**a] will eventually be directed to ~apply(plt.plot), while f"x^{a}" will be directed to aS(plt.legend)
    exps | apply(lambda a: [[x, x**a], f"x^{a}"]) | batched(4) | (sketch() | transpose() | ~apply(plt.plot) + iden() | deref() | rItem(1) | aS(plt.legend)) | deref()

Last line will generate this plot:

.. image:: ../images/probScale.png

Is it worth the extra confusion? Afterall, it just saves you 2-3 lines of
code. To me, it is worth it, because you can quickly change styles (add
a grid, make y axis log)

See also: :class:`~k1lib.cli.output.plotImgs`

Check out a gallery of more examples at `kapi/9-mpl <https://mlexps.com/kapi/9-mpl/>`_.

:param transforms: transform functions to be run when drawing every plot. ``plt`` (aka ``matplotlib.pyplot``) will be passed in
:param titles: if specified, use these titles for each plot. Kinda hacky I have to admit
:param im: if True, returns a PIL image and closes the sketch, else return nothing but still have the sketch open
:param ncols: if specified, will sketch with this number of columns
:param n: if specified, use this number of sketch instead of figuring out automatically
:param axes: if specified, forgo calculating #axes and initialization altogether and just use the provided axes"""
        super().__init__(capture=True); self.titles = titles; self.im = im
        self.transforms = [cli.fastF(t) for t in transforms]; self.ncols = ncols; self.n = n; self.axes = axes
    def __ror__(self, it):
        it = list(it); n = self.n or len(it); s = self.capturedSerial; transforms = self.transforms
        ncols = self.ncols or math.ceil(n**0.5); nrows = math.ceil(n/ncols)
        if self.axes: axes = self.axes
        else:
            fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*5, nrows*4))
            if nrows*ncols == 1: axes = [axes]
        if axes | cli.shape() | cli.shape(0) > 1: axes = axes.flatten()
        for i, [ax, e, title] in enumerate(zip(axes, it, self.titles or ("" | cli.repeat()))):
            plt.sca(ax); e | s | cli.deref()
            if title: plt.title(title)
            for trans in transforms: trans(plt)
        if self.n is None: axes[i+1:] | cli.op().remove().all() | cli.deref(); plt.tight_layout()
        if self.im: return plt.gcf() | cli.toImg()
        if self.n: return axes[i+1:]
    def _jsF(self, meta):
        if self.n: raise Exception("sketch()._jsF() doesn't support .n parameter yet")
        if self.axes: raise Exception("sketch()._jsF() doesn't support .axes parameter yet")
        fIdx = init._jsFAuto(); dataIdx = init._jsDAuto(); ctxIdx = init._jsDAuto()
        # generate all child functions here
        sketch._jsF_ctxIdx = ctxIdx
        header, _fIdx, _async = k1lib.kast.asyncGuard(self.capturedSerial._jsF(meta))
        # then generate all transforms here, using a tracing compiler
        tfStmts = ""
        if len(self.transforms) > 0:
            class Interceptor:
                def __getattr__(self, attr):
                    if getattr(plt, attr) not in sketch_interceptor: raise Exception(f"Transpiling function `plt.{attr}` is not supported at the moment")
                    return lambda *args, **kwargs: sketch_interceptor[getattr(plt, attr)](*args, **kwargs)
            tfStmts = tfs = self.transforms | cli.apply(cli.init.fastF) | cli.op()(Interceptor()).all() | cli.join("; ")
        sketch._jsF_ctxIdx = None
        return f"""\
let {ctxIdx} = null;\n{header}
const {fIdx} = async ({dataIdx}) => {{ // dataIdx should have
    const ctx = []; // this is the object that will be sent to the rendering server!
    const titles = {json.dumps(self.titles)} ?? Array({dataIdx}.length);
    for (const i of [...Array({dataIdx}.length).keys()]) {{
        {ctxIdx} = [];
        // actually executing function and plotting function downstream
        {'await ' if _async else ''}{_fIdx}({dataIdx}[i]);
        if (titles[i]) {ctxIdx}.push(["title", titles[i]]);
        // inject all transforms here
        {tfStmts};
        ctx.push({ctxIdx}); {ctxIdx} = null;
    }}
    // console.log(ctx);
    // console.log(JSON.stringify(ctx));
    const res = await (await fetch("https://local.mlexps.com/routeServer/kapi_9-mpl", {{
      method: "POST",
      body: JSON.stringify({{ "ctx": JSON.stringify(ctx) }}),
      headers: {{ "Content-Type": "application/json" }}
    }})).json()
    if (res.success) {{
        const base64 = res.data;
        console.log("mpl fetched");
        return `<img src="data:image/jpg;base64, ${{base64}}" />`
    }} else {{ throw new Error(res.reason); }}
    // return ctx;
}}""", fIdx
        return f"const {fIdx} = ({dataIdx}) => {dataIdx}.repeatFrom({cli.kjs.v(self.limit)})", fIdx
def _jsF_plt_ctxGuard():
    if sketch._jsF_ctxIdx is None: raise Exception("Have to wrap any plotting operations around sketch(). So, transform your code from `data | (toJsFunc() | ~aS(plt.plot))` into `[data] | (toJsFunc() | (sketch() | ~aS(plt.plot)))`")
    return sketch._jsF_ctxIdx
try: import matplotlib.pyplot as plt; hasMpl = True
except: hasMpl = False
if hasMpl:
    def _jsF_plt_plot(meta, c=None):
        fIdx = init._jsFAuto(); xIdx = init._jsDAuto(); yIdx = init._jsDAuto(); ctxIdx = _jsF_plt_ctxGuard()
        return f"""\
    const {fIdx} = ({xIdx}, {yIdx}=null) => {{
        if (!{yIdx}) {{ // handle only xIdx is available case
            {yIdx} = {xIdx}; {xIdx} = [...Array({yIdx}.length).keys()];
        }}
        {ctxIdx}.push(["plot", {xIdx}, {yIdx}]);
    }}""", fIdx
    settings.kjs.jsF[plt.plot] = _jsF_plt_plot
    def _jsF_plt_title(meta): # version that passes args in js side
        fIdx = init._jsFAuto(); titleIdx = init._jsDAuto(); ctxIdx = _jsF_plt_ctxGuard()
        return f"""const {fIdx} = ({titleIdx}) => {{ {ctxIdx}.push(["title", {titleIdx}]); }}""", fIdx
    settings.kjs.jsF[plt.title] = _jsF_plt_title # below is version that passes args in python side, returns statement, instead of (header, fIdx) like usual
    sketch_interceptor[plt.title] = lambda title: f"""{_jsF_plt_ctxGuard()}.push(["title", `{title}`])"""
    def _jsF_plt_grid(meta):
        fIdx = init._jsFAuto(); tfIdx = init._jsDAuto(); ctxIdx = _jsF_plt_ctxGuard()
        return f"""const {fIdx} = ({tfIdx}) => {{ {ctxIdx}.push(["grid", {tfIdx}]); }}""", fIdx
    settings.kjs.jsF[plt.grid] = _jsF_plt_grid; sketch_interceptor[plt.grid] = lambda tf=True: f"""{_jsF_plt_ctxGuard()}.push(["grid", {cli.kjs.v(tf)}])"""
    def _jsF_plt_legend(meta):
        fIdx = init._jsFAuto(); legendIdx = init._jsDAuto(); ctxIdx = _jsF_plt_ctxGuard()
        return f"""const {fIdx} = ({legendIdx}) => {{ {ctxIdx}.push(["legend", {legendIdx}]); }}""", fIdx
    settings.kjs.jsF[plt.legend] = _jsF_plt_legend; sketch_interceptor[plt.legend] = lambda legend=None: f"""{_jsF_plt_ctxGuard()}.push(["legend", {cli.kjs.v(legend)}])"""

    sketch_interceptor[plt.xlim] = lambda left=None, right=None: f"""{_jsF_plt_ctxGuard()}.push(["xlim", {cli.kjs.v(left)}, {cli.kjs.v(right)}])"""
    sketch_interceptor[plt.ylim] = lambda bottom=None, top=None: f"""{_jsF_plt_ctxGuard()}.push(["ylim", {cli.kjs.v(bottom)}, {cli.kjs.v(top)}])"""
    sketch_interceptor[plt.xscale] = lambda scale: f"""{_jsF_plt_ctxGuard()}.push(["xscale", {cli.kjs.v(scale)}])"""

In [48]:
x = np.linspace(-2, 2); exps = range(1, 9)
exps | cli.apply(lambda a: [[x, x**a], f"x^{a}"]) | cli.batched(4) | (sketch(im=True) | cli.transpose() | ~cli.apply(plt.plot) + cli.iden() | cli.deref() | cli.rItem(1) | cli.aS(plt.legend)) | cli.toBytes() | cli.file("../../docs/images/utils_sketch.png")

'../../docs/images/utils_sketch.png'

In [49]:
#export
import numbers, sys; from collections import deque
class syncStepper(BaseCli):
    def __init__(self, col=0, sort=False):
        """Steps forward all streams at a time, yielding same results from min to max.
That's a bit vague, so let's see an example::

    a = [["a", 1], ["b", 7 ], ["c", 4], ["e", 6]]
    b = [["b", 5], ["c", 1 ], ["d", 3], ["f", 5]]
    c = [["a", 2], ["c", -4], ["d", 9], ["e", 4]]

    [a, b, c] | syncStepper() | deref() # sync-step by the 1st column
    [a, b, c] | syncStepper(1, True) | deref() # sync-step by the 2nd column. Have to sort it explicitly

The first line returns this::

    [[['a', 1], None, ['a', 2]],
     [['b', 7], ['b', 5], None],
     [['c', 4], ['c', 1], ['c', -4]],
     [None, ['d', 3], ['d', 9]],
     [['e', 6], None, ['e', 4]],
     [None, ['f', 5], None]]

The second line returns this::

    [[None, None, ['c', -4]],
     [['a', 1], ['c', 1], None],
     [None, None, ['a', 2]],
     [None, ['d', 3], None],
     [['c', 4], None, ['e', 4]],
     [None, ['b', 5], None],
     [['e', 6], None, None],
     [['b', 7], None, None],
     [None, None, ['d', 9]]]

``col`` can be None, but it's quite a strange use case::

    [['a', 'b', 'c', 'e'], ['b', 'c', 'd', 'f'], ['a', 'c', 'd', 'e']] | syncStepper(None) | deref()

It returns this::

    [[['a'], None, ['a']],
     [['b'], ['b'], None],
     [['c'], ['c'], ['c']],
     [None, ['d'], ['d']],
     [['e'], None, ['e']],
     [None, ['f'], None]]

As you can see, for each line, it kinda yields elements with the same column. If
that element doesn't exist, it'll just put None there. This expects the input
streams are sorted at the column of interest. If they are not, specify ``sort=True``.

It has roughly the same vibe as :class:`~k1lib.cli.structural.groupBy`, in that
it groups everything by a specific column. The main difference here is that you
can sync-step them line-by-line, loading very little into memory, so you can run
this on giant datasets and not have to worry about running out of memory.

With k streams each having n elements, you should expect memory complexity to be
O(k), and the time complexity to be O(n*k^2/2). That k^2 term is kinda worrying,
but in most use cases, k is small and so k^2 can be treated as a constant

See also: :class:`~k1lib.cli.structural.latch`

:param col: column where it should compare values and merge them together. Can be None, but that would be quite a weird use case
:param sort: whether to sort the streams or not. This cli requires it, but it's
    not turned on by default because it's an intensive operation"""
        if col is None: self.col = 0; self.colPreprocess = cli.wrapList().all()
        else: self.col = col; self.colPreprocess = cli.iden()
        self.bank = deque(); self.sentinel = object(); self._sort = sort
    def _append(self, stIdx1, val1, elem1): # append to bank in the correct position
        i = 0; val2 = self.minObj
        for i, [stIdx2, val2, elem2] in enumerate(self.bank):
            if val1 <= val2: break
        if val1 <= val2: self.bank.insert(i, [stIdx1, val1, elem1])
        else: self.bank.append([stIdx1, val1, elem1])
    def _yieldNext(self): # yield the next set of values
        n = len(self.sts); res = [None]*n; last = None; hasInit = False; changed = False; bank = self.bank; sentinel = self.sentinel
        for i, [stIdx, val, elem] in enumerate(bank):
            if not hasInit and elem is sentinel: return res, changed
            if last == val or not hasInit: changed = True; res[stIdx] = elem
            elif hasInit: break
            hasInit = True; last = val
        while bank[0][1] == last: # popping the values off
            stIdx, val1, elem1 = bank.popleft(); val2, elem2 = next(self.sts[stIdx])
            if val1 > val2: raise Exception(f"Stream {stIdx} has not been sorted yet! Please sort all streams before passing it into syncStepper")
            self._append(stIdx, val2, elem2)
        return res, changed
    def __ror__(self, sts): # sts = "streams"
        col = self.col
        # --------------------- All of this is just to figure out the type of the column dynamically. So painful ---------------------
        samples, sts = sts | self.colPreprocess.all() | cli.apply(cli.peek()) | cli.transpose() | cli.cut(col) + cli.iden() | cli.apply(list)
        if len([e for e in sts if e != []]) == 0: return # no elements to yield at all!
        n_nums = sum([1 if isinstance(e, numbers.Number) else 0 for e in samples])
        n_strs = sum([1 if isinstance(e, str) else 0 for e in samples]); n = len(samples)
        if n_nums*(n-n_nums) + n_strs*(n-n_strs) > 0: raise Exception("The requested column in some of the streams is not purely of numeric or string type, a requirement of syncStepper(). Please fix your data structure and try again.")
        if n_nums + n_strs == 0: raise Exception("The requested column in some of the streams is not of numeric or string type, so can't compare them to sync-step them")
        # n = 3; n_strs = 1
        text = n_strs > 0; self.minObj = "" if text else float("-inf"); self.maxObj = chr(sys.maxunicode) if text else float("inf"); senObj = [self.maxObj, self.sentinel]
        # --------------------- And here's the meat of the cli ---------------------
        sts = sts | (cli.sort(col, not text).all() if self._sort else cli.iden()) | cli.apply(lambda st: [st | cli.apply(lambda elem: [elem[col], elem]), senObj | cli.repeat()] | cli.joinStreams()) | cli.aS(list)
        sts | cli.apply(next) | cli.insertIdColumn() | ~cli.apply(lambda idx,e: self._append(idx, *e)) | cli.ignore(); self.sts = sts
        while True:
            res, changed = self._yieldNext()
            if not changed: break
            yield res

In [50]:
a = [["a", 1], ["b", 7],  ["c", 4], ["e", 6]]
b = [["b", 5], ["c", 1],  ["d", 3], ["f", 5]]
c = [["a", 2], ["c", -4], ["d", 9], ["e", 4]]

# yields first element [["a", 1], None, ["a", 2]]
#[a, b, c] | sort(1).all() | syncStepper(1) | deref()
assert [a, b, c] | syncStepper() | deref() == [[['a', 1], None, ['a', 2]], [['b', 7], ['b', 5], None], [['c', 4], ['c', 1], ['c', -4]], [None, ['d', 3], ['d', 9]], [['e', 6], None, ['e', 4]], [None, ['f', 5], None]]
assert [a, b, c] | syncStepper(1, True) | deref() == [[None, None, ['c', -4]], [['a', 1], ['c', 1], None], [None, None, ['a', 2]], [None, ['d', 3], None], [['c', 4], None, ['e', 4]], [None, ['b', 5], None], [['e', 6], None, None], [['b', 7], None, None], [None, None, ['d', 9]]]
assert [['a', 'b', 'c', 'e'], ['b', 'c', 'd', 'f'], ['a', 'c', 'd', 'e']] | syncStepper(None) | deref() == [[['a'], None, ['a']], [['b'], ['b'], None], [['c'], ['c'], ['c']], [None, ['d'], ['d']], [['e'], None, ['e']], [None, ['f'], None]]

In [154]:
#export
class zeroes(BaseCli):
    def __init__(self, col:int=None, log=False, offset:float=0):
        """Shift the specified column so that the first element is zero
Example::

    range(13, 20)   | zeroes()         | deref() # returns [0, 1, 2, 3, 4, 5, 6]
    range(13, 20)   | zeroes(offset=5) | deref() # returns [5, 6, 7, 8, 9, 10, 11]
    [2, 3, 1, 4, 7] | zeroes()         | deref() # returns [0, 1, -1, 2, 5]

Assumes the first element is going to be transformed to zero, thus the last example.
This cli also has log mode, where the natural log of the values will be shifted to zero::

    # returns [1.0, 1.5, 0.5, 2.0, 3.5]
    [2, 3, 1, 4, 7] | zeroes(log=True)           | aS(round, 2).all() | deref()
    # returns [2.72, 4.08, 1.36, 5.44, 9.51]
    [2, 3, 1, 4, 7] | zeroes(offset=1, log=True) | aS(round, 2).all() | deref()

This is essentially the same as dividing everything by 2, so that the first element
turns into 1. Super neat. The 2nd example is equivalent to multiplying everything by e/2.

This cli can function in a table (.col != None)::

    # returns [[0, 'a'], [1, 'b'], [2, 'c'], [3, 'd'], [4, 'e'], [5, 'f'], [6, 'g']]
    [[13, 'a'], [14, 'b'], [15, 'c'], [16, 'd'], [17, 'e'], [18, 'f'], [19, 'g']] | zeroes(0) | deref()

This cli can also act across multiple list of numbers::

    data = [[2, 3, 1, 4, 7], [1, 4, 3, 6, 9]]
    data2 = [[[2, 'b'], [3, 'c'], [1, 'a'], [4, 'd'], [7, 'g']], [[1, 'a'], [4, 'd'], [3, 'c'], [6, 'f'], [9, 'i']]]

    # returns [[0, 1, -1, 2, 5], [5, 8, 7, 10, 13]]
    data | ~zeroes() | deref()
    # returns [[1, 2, 0, 3, 6], [6, 9, 8, 11, 14]]
    data | ~zeroes(offset=1) | deref()
    # returns [[1.0, 1.5, 0.5, 2.0, 3.5], [3.5, 14.0, 10.5, 21.0, 31.5]]
    data | ~zeroes(log=True) | aS(round, 2).all(2) | deref()
    
    # returns [[[0, 'b'], [1, 'c'], [-1, 'a'], [2, 'd'], [5, 'g']], [[5, 'a'], [8, 'd'], [7, 'c'], [10, 'f'], [13, 'i']]]
    data2 | ~zeroes(0) | deref()

So as you can see, the offsets are adjusted so that the first element of each list
starts from the last element of the previous list

:param col: column to shift values
:param offset: custom offset of the minimum value, defaulted to zero
:param log: whether to zero it linearly or zero it logarithmically"""
        self.col = col; self.log = log; self.offset = offset; self.inverted = False
    def __invert__(self): res = zeroes(self.col, self.log, self.offset); res.inverted = True; return res
    def __ror__(self, it):
        col = self.col; log = self.log; offset = self.offset
        if self.inverted:
            def gen():
                currentOffset = offset
                for arr in it:
                    arr = arr | zeroes(col, log, currentOffset)
                    if isinstance(arr, settings.arrayTypes):
                        bm = np if isinstance(arr, np.ndarray) else (torch if hasTorch and isinstance(arr, torch.Tensor) else None)
                        if bm:
                            if col is None: currentOffset = bm.log(arr[-1]) if log else arr[-1]
                            else: currentOffset = bm.log(arr[-1][col]) if log else arr[-1][col]
                            yield arr; continue
                    # yes, we have to deref() them, even though perf will suffer, because let's say
                    # that the user then does rItem(3), and discards elements 0, 1 and 2. Then 0, 1, 2
                    # won't be run, so element 3 won't know its offset!
                    if col is None: arr = list(arr);        currentOffset = math.log(arr[-1])      if log else arr[-1]
                    else: arr = [list(row) for row in arr]; currentOffset = math.log(arr[-1][col]) if log else arr[-1][col]
                    yield arr
            return gen()
        if isinstance(it, settings.arrayTypes):
            bm = np if isinstance(it, np.ndarray) else (torch if hasTorch and isinstance(it, torch.Tensor) else None)
            if bm:
                cloneF = np.copy if isinstance(it, np.ndarray) else torch.clone
                if log:
                    if col is None: minValue = bm.log(it[0]) - offset; return bm.exp(bm.log(it) - minValue)
                    else: minValue = bm.log(it[0, col]) - offset; it = cloneF(it); it[:,col] = bm.exp(bm.log(it[:,col]) - minValue); return it
                else:
                    if col is None: minValue = it[0] - offset; return it - minValue
                    else: minValue = it[0, col] - offset; it = cloneF(it); it[:,col] = it[:,col] - minValue; return it
        row, it = it | cli.peek()
        if it == []: return []
        if log:
            mlog = math.log; mexp = math.exp
            if col is None: minValue = mlog(row) - offset; return (mexp(mlog(row) - minValue) for row in it)
            else: minValue = mlog(row[col]) - offset; return ([*row[:col], mexp(mlog(row[col]) - minValue), *row[col+1:]] for row in it)
        else:
            if col is None: minValue = row - offset; return (row - minValue for row in it)
            else: minValue = row[col] - offset; return ([*row[:col], row[col] - minValue, *row[col+1:]] for row in it)

In [157]:
assert range(13, 20) | zeroes(offset=5) | cli.deref() == [5, 6, 7, 8, 9, 10, 11]
assert range(13, 20) | zeroes() | cli.deref() == [0, 1, 2, 3, 4, 5, 6]
assert [2, 3, 1, 4, 7] | zeroes() | cli.deref() == [0, 1, -1, 2, 5]
assert [2, 3, 1, 4, 7] | zeroes(log=True) | cli.aS(round, 2).all() | cli.deref() == [1.0, 1.5, 0.5, 2.0, 3.5]
assert [2, 3, 1, 4, 7] | zeroes(offset=1, log=True) | cli.aS(round, 2).all() | cli.deref() == [2.72, 4.08, 1.36, 5.44, 9.51]
assert [[13, 'a'], [14, 'b'], [15, 'c'], [16, 'd'], [17, 'e'], [18, 'f'], [19, 'g']] | zeroes(0) | cli.deref() == [[0, 'a'], [1, 'b'], [2, 'c'], [3, 'd'], [4, 'e'], [5, 'f'], [6, 'g']]
data = [[2, 3, 1, 4, 7], [1, 4, 3, 6, 9]]
data2 = data | cli.aS(lambda x: [x, chr(x+97-1)]).all(2) | cli.deref()
data3 = data | cli.aS(lambda x: [x, x+10]).all(2) | cli.deref()
assert data               | ~zeroes() | cli.deref()          == [[0, 1, -1, 2, 5], [5, 8, 7, 10, 13]]
assert np.array(data)     | ~zeroes() | cli.deref(igT=False) == [[0, 1, -1, 2, 5], [5, 8, 7, 10, 13]]
assert torch.Tensor(data) | ~zeroes() | cli.deref(igT=False) == [[0, 1, -1, 2, 5], [5, 8, 7, 10, 13]]
assert data | ~zeroes(offset=1) | cli.deref() == [[1, 2, 0, 3, 6], [6, 9, 8, 11, 14]]
assert data | ~zeroes(log=True) | cli.aS(round, 2).all(2) | cli.deref() == [[1.0, 1.5, 0.5, 2.0, 3.5], [3.5, 14.0, 10.5, 21.0, 31.5]]
assert data2 | ~zeroes(0) | cli.deref() == [[[0, 'b'], [1, 'c'], [-1, 'a'], [2, 'd'], [5, 'g']], [[5, 'a'], [8, 'd'], [7, 'c'], [10, 'f'], [13, 'i']]]
assert data3 | cli.aS(np.array) | ~zeroes(0) | cli.deref(igT=False) == [[[0, 12], [1, 13], [-1, 11], [2, 14], [5, 17]], [[5, 11], [8, 14], [7, 13], [10, 16], [13, 19]]]
x = np.random.randn(100)+10; xT = torch.randn(100)+10
assert x | zeroes()         | shape() == (100,)
assert x | zeroes(log=True) | shape() == (100,)
try: x | zeroes(1); raise Exception("Failed")
except IndexError: pass
try: x | zeroes(1, log=True); raise Exception("Failed")
except IndexError: pass
assert xT | zeroes()         | shape() == (100,)
assert xT | zeroes(log=True) | shape() == (100,)
try: xT | zeroes(1); raise Exception("Failed")
except IndexError: pass
try: xT | zeroes(1, log=True); raise Exception("Failed")
except IndexError: pass
x = np.random.randn(100, 30)+10; xT = torch.randn(100, 30)+10
assert x | zeroes()  | shape() == (100, 30)
assert x | zeroes(3) | shape() == (100, 30)
assert x | zeroes(log=True)    | shape() == (100, 30)
assert x | zeroes(3, log=True) | shape() == (100, 30)
assert xT | zeroes()  | shape() == (100, 30)
assert xT | zeroes(3) | shape() == (100, 30)
assert xT | zeroes(log=True)    | shape() == (100, 30)
assert xT | zeroes(3, log=True) | shape() == (100, 30)

In [55]:
#export
class normalize(BaseCli):
    def __init__(self, col:int=None, mode:int=0):
        """Normalize the data going in.
Example::

    arr = np.random.randn(100)+10
    arr | normalize()       # returns array with mean around 0
    arr | normalize(mode=1) # returns array with mean around 0.5, min 0, max 1

    arr = np.random.randn(100, 20)+10
    arr | normalize(2)         # returns array with 2nd (0-indexing!) column have mean around 0. Other columns not touched
    arr | normalize(2, mode=1) # returns array with 2nd (0-indexing!) column have mean around 0.5

:param col: column to apply the normalization to
:param mode: 0 for ``(x - x.mean())/s.std()``, 1 for ``(x - x.min())/(x.max() - x.min())``"""
        self.col = col; self.mode = mode
    def __ror__(self, x):
        col = self.col; mode = self.mode
        if isinstance(x, k1lib.settings.cli.arrayTypes):
            dims = len(x.shape)
            if col is None: return (x - x.mean())/x.std() if mode == 0 else (x - x.min())/(x.max() - x.min())
            else:
                if mode == 0: xc = x[:,col]; x[:,col] = (xc - xc.mean())/xc.std(); return x
                else: xc = x[:,col]; x[:,col] = (xc - xc.min())/(xc.max() - xc.min()); return x
        if col is None: return np.array(list(x)) | self
        else:
            it = x; ans = []; it = it | cli.deref(2)
            if mode == 0:
                mean = [row[col] for row in it] | cli.toMean()
                std = [row[col] for row in it] | cli.toStd()
                for row in it: row[col] = (row[col]-mean)/std; ans.append(row)
            else:
                _min = min([row[col] for row in it])
                _max = max([row[col] for row in it])
                for row in it: row[col] = (row[col]-_min)/(_max-_min); ans.append(row)
            return ans

In [108]:
x = np.random.randn(100)+10; xT = torch.randn(100)+10
assert abs(x  | normalize()  | cli.toMean()) < 1e-4
assert abs(xT | normalize()  | cli.toMean()) < 1e-4
assert abs(x  | cli.deref(igT=False) | normalize()  | cli.toMean()) < 1e-4
assert abs(xT | cli.deref(igT=False) | normalize()  | cli.toMean()) < 1e-4
x = np.random.randn(100, 30)+10; xT = torch.randn(100, 30)+10
assert abs(x  | normalize()  | cli.toMean()) < 1e-4
assert abs(xT | normalize()  | cli.toMean()) < 1e-4
assert abs(x  | normalize(2) | cli.toMean()) > 8
assert abs(xT | normalize(2) | cli.toMean()) > 8
assert (x  | normalize(2) | cli.op()[:,2] | cli.toMean()) < 1e-4
assert (xT | normalize(2) | cli.op()[:,2] | cli.toMean()) < 1e-4
assert (x  | cli.deref(igT=False) | normalize(2) | cli.toMean().all() | cli.toMean()) > 8
assert (xT | cli.deref(igT=False) | normalize(2) | cli.toMean().all() | cli.toMean()) > 8

In [49]:
!../../export.py cli/utils --upload=True

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-12-29 07:54:46,023	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2023-12-29 07:54:46,040	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- exportAll
13302   0   60%   
8861    1   40%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.4.5
Uninstalling k1lib-1.4.4.5:
  Successfully uninstalled k1lib-1.4.4.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/

In [51]:
!../../export.py cli/utils --bootstrap=True

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-12-31 16:00:57,162	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2023-12-31 16:00:57,170	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- bootstrapping
Current dir: /home/kelvin/repos/labs/k1lib, /home/kelvin/repos/labs/k1lib/k1lib/cli/../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.4.5
Uninstalling k1lib-1.4.4.5:
  Successfully uninstalled k1lib-1.4.4.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-b